In [5]:
from flask import Flask
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
# 1) https://developers.kakao.com/ 로그인
# 2) 내 애플리케이션 -> 이전에 만들었던 사항 클릭
# 3) 동의항목: 닉네임 - 필수, 카카오계정 - 선택, 카톡 메시지 - 선택
# 4) 카카오 로그인 - 활성화 설정 ON
# 5) Redirect URI에 https://localhost.com 설정
# 6) 플랫폼 - Web 도메인으로 https://www.naver.com과 https://localhost.com 설정
# 7) Kakao API 문서 위치 - https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api
# 8) 주어진 정보 확인
#    GET /oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code HTTP/1.1
#    Host: kauth.kakao.com
#    요약 정보에 있는 REST API 키가 REST_API_KEY 위치에 배치되면 됨
#    REDIRECT_URI의 경우 앞서 등록했던 https://localhost.com
# 9) 해당 URL로 요청을 하면 Kakao API를 활용할 수 있는 토큰을 발급받을 수 있고
#    주어진 시간내에 처리하지 않으면 토큰이 만료되므로 먼저 코드를 작성하도록 한다.

In [7]:
import requests

url = "https://kauth.kakao.com/oauth/token"


data = {
    "grant_type" : "authorization_code",
    "client_id" : "b3768d46a42cd7d9ca7867c1dc8750c0",
    "redirect_uri" : "https://localhost.com",
    "code" : "E2Psvrk7bpqdfTUhUvycyLw1C61qB7yj0yloGWkrsW8QAfXQaRt2UzZ3lWYmpD0RgGyq7gopcJ4AAAF9VkbqTA"
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

{'access_token': '8QzIJiwAy2Jp6Y8MNcZKJnLCVFHz9s5mewnFuAorDSAAAAF9Vke4OA', 'token_type': 'bearer', 'refresh_token': 'dZex6pevO5fDRgzQcfOEd41dFLv8ILgpTCHktgorDSAAAAF9Vke4Ng', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname friends', 'refresh_token_expires_in': 5183999}


In [16]:
import json
import requests
from datetime import datetime
import os


# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = "res/kakao_login/kakao_token.json"

# 저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# Access Token 갱신을 위한 Refresh
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [21]:

# 토큰 정보 활용
tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

# 메시지 보내기 URL
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# requests parameter 설정
headers = {
    # Authorization: Bearer {ACCESS_TOKEN}
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "저희 사이트에 방문해주셔서 감사합니다.",
        "link" : {
            # 이동할 링크 정보니까 사이트 주소를 걸면 더 좋음
            
        }
    })
}

# 메시지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print("Error!")
else:
    print("카톡 메시지 전송 완료")

200
카톡 메시지가 간드아!!!
